# Setup

## Libraries + dataset 

In [68]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf
import openpyxl

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
        pathPart1 = "/Users/"
        pathPart2 = "/documents/Github/SCGProject/Datasets/"
        complePath = pathPart1+username+pathPart2

        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")

else:
    complePath = "C:/SCGProject/Datasets/"

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(
    ',', '.')


Username:  marcovinciguerra


## Removing the intermediate in the dataset

### Budget

In [69]:
dfConsumi.rename(columns={'Budget/cons': 'budget'}, inplace=True)
qtaArtBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfConsumi where budget = "BUDGET" group by NrArticolo')

dfVendite.rename(columns={'budget/cons': 'budget'}, inplace=True)
qtaArtVendBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfVendite where budget = "BUDGET" group by NrArticolo')

artProdeVendBudget = sqldf(
    'select P.NrArticolo from qtaArtBudget P join qtaArtVendBudget V on P.NrArticolo = V.NrArticolo')


consumiB = sqldf('select * from dfConsumi where budget = "BUDGET"')


# ho trovato articoli che hanno consumato risorse a budget ma non sono stati messi tra le vendite a budget
artProdeNONVendBudget = sqldf(
    'select * from consumiB group by NrArticolo having NrArticolo not in (select NrArticolo from artProdeVendBudget)')

# prendo gli articoli che usano come materia prima almeno un articolo prodotto internamente
# sqldf('select NrMovimento, budget, C.NrArticolo, CodiceMP, NrDocumento, QuantityMPImpiegata, ImportoCostoTOTALE from consumiB C join artProdeNONVendBudget A on C.CodiceMP = A.NrArticolo')

costoPerArticoloBudget = sqldf(
    'select NrArticolo, sum(ImportoCostoTOTALE) as costoMpPerArt from consumiB where NrArticolo not in (select NrArticolo from artProdeNONVendBudget) group by NrArticolo')
sqldf('select sum(costoMpPerArt) as costoTotBudget from costoPerArticoloBudget')

mpB = sqldf('''select NrMovimento,budget,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiB group by NrArticolo''')
prodInternaB = sqldf('''select A.NrMovimento, A.budget, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiB A join mpB B on A.CodiceMP = B.NrArticolo''')

mpIntermediB = sqldf('''select distinct CodiceMP from prodInternaB''')
consumiBNOArtInterm = sqldf(
    '''select * from consumiB where NrArticolo not in (select CodiceMP from mpIntermediB)''')


### Consuntivo

In [70]:
consumiC = sqldf(
    '''select * from dfConsumi where budget = "CONSUNTIVO" or budget = "Consuntivo"''')
mpC = sqldf('''select NrMovimento,budget,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiC group by NrArticolo''')
#A.NrArticolo, A.CodiceMP, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE, A.CodiceMP
prodInternaC = sqldf('''select A.NrMovimento, A.budget, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiC A join mpC B on A.CodiceMP = B.NrArticolo''')
mpIntermediC = sqldf('''select distinct CodiceMP from prodInternaC''')
consumiCNOArtInterm = sqldf(
    '''select * from consumiC where NrArticolo NOT IN (select CodiceMP from mpIntermediC)''')


### UNIONE DELLE DUE TABELLE PER FARE IL NUOVO DATASET

In [71]:
dfConsumi = sqldf('''  SELECT *
                            FROM consumiBNOArtInterm
                            UNION
                            SELECT *
                            FROM consumiCNOArtInterm
                            ''')
del dfConsumi['Unnamed: 0']
dfConsumi


,NrMovimento,budget,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
...,...,...,...,...,...,...,...
4046,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
4047,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
4048,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
4049,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00


# Funzione che fa partire l'analisi degli scostamenti

## Reading delle tabelle che servono 

In [72]:
venditeS1   = pd.read_csv("venditeS1.csv")
MergeImpiego = pd.read_csv("MergeImpieg.csv")
ConsumiUnit = pd.read_csv("ConsumiUnit.csv")


In [73]:
venditeS1


,Unnamed: 0,NrArticolo,Mix_x,VolumeTotale_x,PrezzoUnitario_x,Valuta,TassoCambioMedio_x,PrezzoUnitarioEuro_x,Mix_y,VolumeTotale_y,PrezzoUnitario_y,TassoCambioMedio_y,PrezzoUnitarioEuro_y
0,0,ART0000018,21438525,9329,55.000000,2,1.1993,45.860085,27685492,7224,45.000000,1.0541,42.690447
1,1,ART0000041,42877050,9329,120.000000,2,1.1993,100.058367,55370985,7224,95.000000,1.0541,90.124277
2,2,ART0000042,42877050,9329,180.000000,2,1.1993,150.087551,55370985,7224,150.000000,1.0541,142.301489
3,3,ART0000043,21438525,9329,150.000000,2,1.1993,125.072959,27685492,7224,115.000000,1.0541,109.097809
4,4,ART0000115,32157787,9329,365.000000,1,1.0000,365.000000,166112956,7224,163.750000,1.0000,163.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,410,ART0005111,75034837,9329,35.000000,2,1.1993,29.183690,27685492,7224,35.000000,1.0541,33.203681
411,411,ART0005112,75034837,9329,45.000000,2,1.1993,37.521888,27685492,7224,45.000000,1.0541,42.690447
412,412,ART0005136,686032800,9329,39.109375,1,1.0000,39.109375,1204318936,7224,102.678161,1.0000,102.678161
413,413,ART0005136,439489763,9329,229.121951,2,1.1993,191.046403,1107419712,7224,52.190125,1.0541,49.511550


In [74]:
MergeImpieg


,Unnamed: 0,NrArticolo,Descrizione,NrAreaProduzione,TempoRisorsa_x,ImpiegoUnitario_x,CostoOrario_x,QuantitydiOutput_x,TempoRisorsa_y,ImpiegoUnitario_y,CostoOrario_y,QuantitydiOutput_y
0,0,ART0000018,Controllo qualità,CQ,0.0000,0.000000,15.0,2.0,0.00,0.000000,12.75,2.0
1,1,ART0000018,Fresatura,A30,1.0000,0.500000,50.0,2.0,0.75,0.375000,51.00,2.0
2,2,ART0000018,Tornitura,A20,1.0000,0.500000,80.0,2.0,1.00,0.500000,68.00,2.0
3,3,ART0000041,Controllo qualità,CQ,0.0000,0.000000,15.0,4.0,0.00,0.000000,12.75,4.0
4,4,ART0000041,Fresatura,A30,0.0001,0.000025,80.0,4.0,1.25,0.312500,51.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1345,ART0004904,Tornitura,A20,NaN,NaN,NaN,NaN,1.75,0.437500,68.00,4.0
1346,1346,ART0004944,Tornitura,A20,NaN,NaN,NaN,NaN,1.75,0.218750,68.00,8.0
1347,1347,ART0005091,Fresatura,A30,NaN,NaN,NaN,NaN,1.00,0.250000,42.50,4.0
1348,1348,ART0005136,Foratura/Maschiatura,A32,NaN,NaN,NaN,NaN,3.00,NaN,42.50,0.0


In [75]:
ConsumiUnit


,Unnamed: 0,CodiceMP,NrArticolo,ImpiegoPerPezzo_x,CostoPerUnitaMis_x,QuantitydiOutput_x,ImportoCostoTOTALE_x,ImpiegoPerPezzo_y,CostoPerUnitaMis_y,QuantitydiOutput_y,ImportoCostoTOTALE_y
0,0,ART0000771,ART0005136,0.009804,230.950000,102.0,230.95,NaN,NaN,NaN,NaN
1,1,ART0000809,ART0005136,0.019608,561.350000,102.0,1122.70,NaN,NaN,NaN,NaN
2,2,ART0001066,ART0003099,56.000000,0.001870,19.0,1.99,NaN,NaN,NaN,NaN
3,3,ART0001068,ART0004671,38.000000,0.036000,5.0,6.84,NaN,NaN,NaN,NaN
4,4,ART0001073,ART0002240,46.000000,0.086685,4.0,15.95,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1134,1134,ART0005913,ART0002191,NaN,NaN,NaN,NaN,20.0,0.339250,2.0,13.57
1135,1135,ART0005917,ART0001610,NaN,NaN,NaN,NaN,243.0,0.000597,4.0,0.58
1136,1136,ART0005930,ART0000229,NaN,NaN,NaN,NaN,130.0,0.005064,12.0,7.90
1137,1137,ART0005935,ART0004675,NaN,NaN,NaN,NaN,45.0,0.014311,5.0,3.22


## By articolo full stack!

In [76]:
def by_articolo_full(venditeS1, mergeImpiego, ConsumiUnit, articolo):

        venditeSingoloArticolo = sqldf('select * from venditeS1 where NrArticolo = "'+articolo+'"')

        BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeSingoloArticolo ').iloc[0][0]

        if(BudgetStandardRic is None):
                BudgetStandardRic = 0

        if(MixStandardRic is None):
                MixStandardRic = 0

        if(MixEffettivoRic is None):
                MixEffettivoRic = 0

        if(BudgetEffettivoRic is None):
                BudgetEffettivoRic = 0

        if(BudgetEffettivoValutaRic is None):
                BudgetEffettivoValutaRic = 0

        ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
        ScostamentoMixRic = MixEffettivoRic-MixStandardRic
        ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
        ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic


        lavorazioniSingoloProdotto = sqldf('select * from MergeImpieg where NrArticolo = "'+articolo+'"')

        #Budget Standard
        BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from lavorazioniSingoloProdotto').iloc[0][0]
        # Impiego Standard (volume a consuntivo)
        ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
        # Impiego Effettivo (volume e impiego a consuntivo)
        ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
        # Budget Effettivo (tutto a consuntivo)
        BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]

        if(BudgetStandardLav is None):
                        BudgetStandardLav = 0

        if(ImpiegoStandardLav is None):
                        ImpiegoStandardLav = 0

        if(ImpiegoEffettivoLav is None):
                        ImpiegoEffettivoLav = 0

        if(BudgetEffettivoLav is None):
                        BudgetEffettivoLav = 0

        #Scostamenti
        ScostamentoVolumeLav = ImpiegoStandardLav-BudgetStandardLav
        ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
        ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav

        consumiSingoloArticolo = sqldf('select * from ConsumiUnit where NrArticolo="'+articolo+'"')

        BudgetEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_x) from consumiSingoloArticolo').iloc[0][0]
        ImpiegoEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
        ImpiegoStandardMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
        BudgetStandardMP = sqldf('select sum(QuantitydiOutput_y*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]

        #TODO: choose strategy 

        BudgetEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_x)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        ImpiegoEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        ImpiegoStandardMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        BudgetStandardMP = sqldf('select (avg(QuantitydiOutput_y)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]

        if(BudgetStandardMP is None):
                BudgetStandardMP = 0

        if(ImpiegoStandardMP is None):
                ImpiegoStandardMP = 0

        if(ImpiegoEffettivoMP is None):
                ImpiegoEffettivoMP = 0

        if(BudgetEffettivoMP is None):
                BudgetEffettivoMP = 0

        ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
        ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
        ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

        data = {articolo: ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
                'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
                'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
                'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
                'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
                'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
                'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
                'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
                'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
                'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
                'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

        return pd.DataFrame(data)


In [77]:
by_articolo_full(venditeS1,MergeImpiego,ConsumiUnit,'ART0002240')

,ART0002240,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,160.516075,46.772749,207.288824,-46.772745,160.516080,10.245707,170.761787,-20.674236,150.087551,-10.428525
1,CostiMP,7.800000,7.800000,15.600000,1.088372,16.688372,-0.738372,15.950000,0.000000,15.950000,8.150000
2,CostiLav,242.250000,242.250000,484.500000,-238.000000,246.500000,-21.500000,225.000000,0.000000,225.000000,-17.250000
3,Margine,-89.533925,-203.277251,-292.811176,190.138883,-102.672292,32.484079,-70.188213,-20.674236,-90.862449,-1.328525


In [80]:
articoli = sqldf('select distinct NrArticolo from dfImpiegoRisorse')
scos = []

for articolo in articoli.values:
   theArticolo = articolo[0]
   print(theArticolo)
   scos.append(by_articolo_full(venditeS1, MergeImpiego, ConsumiUnit, theArticolo))

ART0000018
ART0000041
ART0000042
ART0000043
ART0000115
ART0000116
ART0000119
ART0000121
ART0000125
ART0000128
ART0000129
ART0000131
ART0000151
ART0000152
ART0000157
ART0000223
ART0000226
ART0000227
ART0000228
ART0000229
ART0000308
ART0000312
ART0000313
ART0000320
ART0000323
ART0000324
ART0000325
ART0000456
ART0000555
ART0000625
ART0000627
ART0000628
ART0000630
ART0000635
ART0000638
ART0000639
ART0000641
ART0000642
ART0000643
ART0000644
ART0000645
ART0000690
ART0000697
ART0000748
ART0000749
ART0000752
ART0000753
ART0000756
ART0000757
ART0000758
ART0000762
ART0000765
ART0000769
ART0000772
ART0000778
ART0000779
ART0000780
ART0000782
ART0000783
ART0000784
ART0000786
ART0000788
ART0000790
ART0000795
ART0000796
ART0000797
ART0000804
ART0000805
ART0000811
ART0000812
ART0000814
ART0000825
ART0000826
ART0000830
ART0000841
ART0000843
ART0000845
ART0000847
ART0000848
ART0000849
ART0000850
ART0000851
ART0000853
ART0000877
ART0000878
ART0000879
ART0000880
ART0000881
ART0000882
ART0000883
ART0000885

IndexError: single positional indexer is out-of-bounds

In [ ]:
def Func(x):
  return x.iloc[0, 10]


scos.sort(key=Func)
scos